In [89]:
import pandas as pd
import opendatasets as od


In [90]:
# od.download("https://www.kaggle.com/competitions/spaceship-titanic")

In [91]:
raw_data=pd.read_csv('spaceship-titanic/train.csv')
raw_data=raw_data[~raw_data['Cabin'].isnull()]





In [92]:
extract_cabin=raw_data.Cabin.str.split('/',expand=True)
raw_data=raw_data.assign(Deck=extract_cabin[0])

In [93]:
ytrain=raw_data['Transported']

In [94]:


def clean_dt(raw_data):
    return (raw_data
    .drop(columns=['Name','PassengerId','Destination','HomePlanet','VIP','Age'])
    .assign(
        RoomService=raw_data.RoomService.fillna(0).replace('nan',0),
        CryoSleep=raw_data.CryoSleep.fillna(False),
        # VIP=raw_data.VIP.fillna(False).replace('nan',0),
        FoodCourt=raw_data.FoodCourt.fillna(0).replace('nan',0),
        ShoppingMall=raw_data.ShoppingMall.fillna(0).replace('nan',0),
        Spa=raw_data.Spa.fillna(0).replace('nan',0),
        VRDeck=raw_data.VRDeck.fillna(0).replace('nan',0),
        Side=raw_data.Cabin.str.split('/',expand=True)[2]=='S',
        Age=raw_data.Age.fillna(raw_data.Age.median())
        )
     
    )



In [95]:
 # encoding boolean labels
from sklearn import preprocessing

def transform_step(raw_data):
    label_encoder=preprocessing.LabelEncoder()
    raw_data.CryoSleep=label_encoder.fit_transform(raw_data.CryoSleep).astype(int)
    raw_data.Side=label_encoder.fit_transform(raw_data.Side).astype(int)
    # raw_data.VIP=label_encoder.fit_transform(raw_data.VIP)
    # raw_data=pd.get_dummies(raw_data,columns=['Destination','HomePlanet'])
    raw_data=pd.get_dummies(raw_data,columns=['Deck'])

    return raw_data

In [96]:
res_encoder=preprocessing.LabelEncoder()

ytrain=res_encoder.fit_transform(ytrain)

In [97]:
  
Xtrain=clean_dt(raw_data)
Xtrain=transform_step(Xtrain)
Xtrain.drop(columns=['Transported'],inplace=True)   

# cabins=Xtrain.Cabin.str.split('/',expand=True),
# cabins

In [98]:
raw_test=pd.read_csv('spaceship-titanic/test.csv')

extract_cabin=raw_test.Cabin.str.split('/',expand=True)
raw_test=raw_test.assign(Deck=extract_cabin[0])

Xtest=clean_dt(raw_test)
Xtest=transform_step(Xtest)

In [99]:
# Xtrain.Cabin1=res_encoder.fit_transform(Xtrain.Cabin1).astype(int)
# Xtest.Cabin1=res_encoder.fit_transform(Xtest.Cabin1).astype(int)

In [100]:
Xtrain.dtypes

CryoSleep         int32
Cabin            object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Side              int32
Age             float64
Deck_A            uint8
Deck_B            uint8
Deck_C            uint8
Deck_D            uint8
Deck_E            uint8
Deck_F            uint8
Deck_G            uint8
Deck_T            uint8
dtype: object

In [101]:
(Xtrain
 .assign(Transported=ytrain==True)
 .corr(method='spearman')
 .style
 .background_gradient(cmap='RdBu',vmax=1,vmin=-1)
 .set_sticky(axis='index')
)

,CryoSleep,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Side,Age,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Transported
CryoSleep,1.000000,-0.507133,-0.521453,-0.504342,-0.537715,-0.515993,0.020889,-0.072007,-0.030365,0.126918,0.029940,-0.066703,-0.110994,-0.221709,0.248042,-0.017717,0.459200
RoomService,-0.507133,1.000000,0.180158,0.428001,0.239585,0.179149,-0.021415,0.123040,-0.042669,-0.147660,-0.096716,0.160788,0.097877,0.235093,-0.218216,0.006157,-0.362576
FoodCourt,-0.521453,0.180158,1.000000,0.186424,0.468248,0.494490,0.008081,0.199336,0.135698,0.064883,0.177280,0.032745,0.026422,-0.002854,-0.233824,0.037322,-0.184313
ShoppingMall,-0.504342,0.428001,0.186424,1.000000,0.255240,0.187024,-0.015791,0.099145,-0.050975,-0.127907,-0.093481,0.106905,0.097780,0.228772,-0.195107,-0.009665,-0.227038
Spa,-0.537715,0.239585,0.468248,0.255240,1.000000,0.435542,0.000925,0.191256,0.105433,0.013277,0.117535,0.064321,0.043096,0.045519,-0.229351,0.031632,-0.365141
VRDeck,-0.515993,0.179149,0.494490,0.187024,0.435542,1.000000,-0.010559,0.176268,0.103968,0.036905,0.144203,0.026496,0.026329,0.032159,-0.216479,0.034345,-0.347117
Side,0.020889,-0.021415,0.008081,-0.015791,0.000925,-0.010559,1.000000,0.010176,0.013448,0.026711,0.024023,-0.011554,0.003694,-0.027308,-0.004543,-0.014797,0.103775
Age,-0.072007,0.123040,0.199336,0.099145,0.191256,0.176268,0.010176,1.000000,0.088467,0.109107,0.143197,0.084742,0.029308,-0.030916,-0.221387,0.020036,-0.071239
Deck_A,-0.030365,-0.042669,0.135698,-0.050975,0.105433,0.103968,0.013448,0.088467,1.000000,-0.056016,-0.054740,-0.043047,-0.059778,-0.123420,-0.115753,-0.004278,-0.002664
Deck_B,0.126918,-0.147660,0.064883,-0.127907,0.013277,0.036905,0.026711,0.109107,-0.056016,1.000000,-0.098672,-0.077595,-0.107754,-0.222473,-0.208653,-0.007712,0.146571


It seems that the persons who have billed luxury amenities have a negative relationship with being transported. In simple words, the people who have bills are less likely to be transported.

In [102]:
# prediction
import xgboost as xgb

In [103]:
# random_for=xgb.XGBClassifier()
model1=xgb.XGBClassifier()


model1.fit(Xtrain,ytrain)
res=model1.predict(Xtest)

ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:Cabin

In [ ]:
# submission
result=pd.DataFrame({'PassengerId':raw_test.PassengerId,'Transported':res_encoder.inverse_transform(res)})
result.to_csv('submission.csv',index=False)